In [26]:
%reset -f

## Lets import some useful packages

In [27]:
import warnings
warnings.filterwarnings("ignore")
import random
from copy import deepcopy as dp
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tqdm import *
import os 
from os import system
import numpy as np 
from keras.preprocessing import image
from keras.initializers import RandomNormal as RN
from PIL import Image
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from matplotlib import pyplot as plt
from datetime import datetime

## Some parameters

In [28]:
size = 128
#img_width, img_height = size, size

data_train_path_target = "data_airbus_defi/train/"
data_test_path = "data_airbus_defi/test/"
input_shape = (size, size, 3)
#imput_shape = (size,size,1)

## Loading data. Numpy arrays

In [29]:
PATH = os.getcwd()
full = True
some = True
if full:
    training_size = float("inf")
    testing_size = float("inf")
elif some:
    training_size = 1000
    testing_size = float("inf")
else:
    training_size = 10
    testing_size = 10
    

train_path = PATH +"/" + data_train_path_target + "target/"
train_data_target = os.listdir(train_path)
x_train = []




#im_train = []
tdt = 0
for sample in (train_data_target):
    img_path = train_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_train.append(np.array(x))
    
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_train.append(im_array)
    """
    tdt += 1
    
    if tdt > training_size:
        break

train_path = PATH +"/" + data_train_path_target + "other/"
train_data_other = os.listdir(train_path)
x_train2 = []
im_train2 = []
tdo = 0
for sample in (train_data_other):
    img_path = train_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_train2.append(np.array(x))
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_train2.append(im_array)
    """
    tdo += 1
    if tdo > training_size:
        break

    
test_path = PATH+'/data_airbus_defi/test/'
test_data = os.listdir(test_path)
x_test = []
#im_test = []


td = 0
output = pd.DataFrame(columns=["name"])
test_data = [str(x) + ".jpg" for x in range(len(test_data))]
for sample in (test_data):
    output.append({"name": sample},ignore_index=True)
    #print(sample)
    img_path = test_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_test.append(np.array(x))
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_test.append(im_array)
    """
    td+=1
    if td > testing_size:
        break
    
    
#test = np.array([np.array(x_t) for x_t in x_test])
# finally converting list into numpy array
x_train = np.array(x_train)
x_train2 = np.array(x_train2)
x_test = np.array(x_test) / 255.
XTRAIN = np.concatenate((x_train, x_train2), axis=0) / 255.

In [30]:
system('say Data chargée!')

0

In [31]:
print("Input data shape")
print(x_train.shape)
print(x_train2.shape)
print(XTRAIN.shape)

Input data shape
(32966, 128, 128, 3)
(35252, 128, 128, 3)
(68218, 128, 128, 3)


In [32]:
#im_train = np.array(im_train)
#im_train2 = np.array(im_train2)
#IMTRAIN = np.concatenate((im_train, im_train2), axis=0).reshape((len(IMTRAIN),size,size,1)) / 255.
#im_test = np.array(im_test).reshape((len(im_test),size,size,1)) / 255.

In [33]:
#print(im_train.shape)
#print(im_train2.shape)
#print(IMTRAIN.shape)
#print(im_test.shape)

## Lablelling data

In [34]:
train_label1 = np.array([1] * x_train.shape[0] + [0] * x_train2.shape[0])

Suffling data. Not (totally) sure it is useful.

Quite ong when we use every data

In [35]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
XTRAIN, train_labels = unison_shuffled_copies(XTRAIN,train_label1)
#IMTRAIN, im_labels = unison_shuffled_copies(IMTRAIN,train_labels)
#XTRAIN, train_labels  = zip(*random.shuffle(list(zip(XTRAIN,train_label1))))

In [36]:
system('say Data mélangée!')

0

## Useful (?) functions

In [38]:
def plotModel(model):
    SVG(model_to_dot(model).create(prog='dot', format='svg'))
def plotLearning(history):
    plt.plot(history.history['acc'])
    #plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    plt.close()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    plt.close()
    
    
def saveModel(model, name):
    path = "model/" + name 
    model_json = model.to_json()
    with open(path + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(path + ".h5")
    print("Saved model")

def loadModel( name):
    path = "model/" + name 
    json_file = open(path + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(path + ".h5")
    #loaded_model.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy'])
    print("Loaded model from disk")
    return loaded_model
    
    

    
def soumissionCSV(prediction, name):
    global test_data
    X = pd.DataFrame()
    X["name"] = test_data
    X["prediction"] = prediction
    X.set_index("name", inplace=True)
    X.to_csv(name + ".csv", sep=";")
    print("CSV file written")
    
def cut_half(x):
    if x < .5:
        return 0
    else:
        return 1
    
def prediction_from_model(model):
    global x_test
    pred = model.predict(x_test)
    prediction = [cut_half(x) for x in pred]
    print("some predictions")
    print(prediction[:9])
    return prediction

## The model. Finally !

In [39]:
trained = False

epochs = 50
validation_split = .1
conv2Dsize = 32
pool_size = (2,2)


modelBW = Sequential()
modelBW.add(Conv2D(conv2Dsize, (3, 3), input_shape = input_shape, activation = 'relu'))
modelBW.add(MaxPooling2D(pool_size = pool_size))
modelBW.add(Conv2D(conv2Dsize, (3, 3), activation = 'relu'))
modelBW.add(MaxPooling2D(pool_size = pool_size))
modelBW.add(Flatten())
modelBW.add(Dense(units = 128, activation = 'relu'))
modelBW.add(Dense(units = 1, activation = 'sigmoid'))

modelBW.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])





Train on 61396 samples, validate on 6822 samples
Epoch 1/50
61396/61396 [==============================] - 1385s 23ms/step - loss: 0.2275 - acc: 0.9135 - val_loss: 0.1784 - val_acc: 0.9342
Epoch 2/50
61396/61396 [==============================] - 1226s 20ms/step - loss: 0.1372 - acc: 0.9488 - val_loss: 0.1344 - val_acc: 0.9494
Epoch 3/50
61396/61396 [==============================] - 1272s 21ms/step - loss: 0.1068 - acc: 0.9609 - val_loss: 0.1329 - val_acc: 0.9516
Epoch 4/50
61396/61396 [==============================] - 1248s 20ms/step - loss: 0.0853 - acc: 0.9691 - val_loss: 0.1243 - val_acc: 0.9565
Epoch 5/50
61396/61396 [==============================] - 1335s 22ms/step - loss: 0.0633 - acc: 0.9782 - val_loss: 0.1279 - val_acc: 0.9582
Epoch 6/50
 1056/61396 [..............................] - ETA: 17:48 - loss: 0.0380 - acc: 0.9848

KeyboardInterrupt: 

In [47]:
if not trained:
    history = modelBW.fit(XTRAIN, train_labels,
            epochs=epochs, verbose=1, validation_split=validation_split)
    plotLearning(history)

Train on 61396 samples, validate on 6822 samples
Epoch 1/50
61396/61396 [==============================] - 2927s 48ms/step - loss: 0.0464 - acc: 0.9851 - val_loss: 0.1622 - val_acc: 0.9563
Epoch 2/50
61396/61396 [==============================] - 4017s 65ms/step - loss: 0.0310 - acc: 0.9903 - val_loss: 0.1725 - val_acc: 0.9582
Epoch 3/50
61396/61396 [==============================] - 1261s 21ms/step - loss: 0.0275 - acc: 0.9913 - val_loss: 0.1939 - val_acc: 0.9550
Epoch 4/50
61396/61396 [==============================] - 1250s 20ms/step - loss: 0.0171 - acc: 0.9947 - val_loss: 0.2182 - val_acc: 0.9551
Epoch 5/50
  480/61396 [..............................] - ETA: 19:03 - loss: 0.0109 - acc: 0.9958

KeyboardInterrupt: 

## Prediction

In [48]:
prediction = prediction_from_model(modelBW)

some predictions
[1, 0, 1, 0, 1, 1, 0, 1, 1]


## Writting output

In [50]:
from datetime import datetime
today = str(datetime.today())[:16]

In [51]:
soumissionCSV(prediction, today)

CSV file written


In [52]:
from os import system
system('say On est bon!')

0

In [53]:
print("Proportion of windmills")
sum(prediction)/ len(prediction)

Proportion of windmills


0.4744957786116323

In [54]:
saveModel(modelBW, today)

Saved model


In [119]:
plotModel(modelBW)

In [55]:
plotLearning(history)

NameError: name 'history' is not defined

In [22]:
loadedModel = loadModel("1erNovembre3")
pred = prediction_from_model(loadedModel)

Loaded model from disk
some predictions
[1, 0, 1, 0, 1, 1, 0, 1, 1]


## One model that was good

In [ ]:
epochs = 50
validation_split = .1
conv2Dsize = 12
pool_size = (2,2)


modelBW = Sequential()
modelBW.add(Conv2D(conv2Dsize, (5), 
                  padding="same", 
                  input_shape=input_shape))
modelBW.add(Activation("relu"))
modelBW.add(MaxPooling2D(pool_size=pool_size, strides=(2, 2)))

# second set of CONV => RELU => POOL layers
modelBW.add(Conv2D(conv2Dsize, (5, 5), padding="same"))
modelBW.add(Activation("relu"))
modelBW.add(MaxPooling2D(pool_size=pool_size, strides=(2, 2)))

#first (and only) set of FC => RELU layers
modelBW.add(Flatten())
modelBW.add(Dense(2))
modelBW.add(Activation("sigmoid"))

# softmax classifier
modelBW.add(Dense(1))
modelBW.add(Activation("relu"))
modelBW.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [31]:
trained = True

modelSMALL = Sequential()
inputShape = (size, size, 3)
modelSMALL.add(Conv2D(20, (5, 5), 
                  padding="same", 
                  input_shape=inputShape,
                  kernel_initializer=RN(0,1)))
modelSMALL.add(Activation("relu"))
modelSMALL.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
modelSMALL.add(Conv2D(50, (5, 5), padding="same"))
modelSMALL.add(Activation("relu"))
modelSMALL.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first (and only) set of FC => RELU layers
modelSMALL.add(Flatten())
modelSMALL.add(Dense(32))
modelSMALL.add(Activation("sigmoid"))

# softmax classifier
modelSMALL.add(Dense(1))
modelSMALL.add(Activation("relu"))
if trained:
    modelSMALL.load_weights('model/modelSMALL.h5')
modelSMALL.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])



if not trained:
    history = modelSMALL.fit(
            XTRAIN, train_labels,
            epochs=4, verbose=1, validation_split = .1)

Train on 7201 samples, validate on 801 samples
Epoch 1/4
7201/7201 [==============================] - 226s 31ms/step - loss: 0.3956 - acc: 0.4958 - val_loss: 0.2504 - val_acc: 0.5069
Epoch 2/4
7201/7201 [==============================] - 229s 32ms/step - loss: 0.2648 - acc: 0.4967 - val_loss: 0.2495 - val_acc: 0.5094
Epoch 3/4
7201/7201 [==============================] - 251s 35ms/step - loss: 0.2493 - acc: 0.5101 - val_loss: 0.2499 - val_acc: 0.5094
Epoch 4/4
7201/7201 [==============================] - 234s 33ms/step - loss: 0.2494 - acc: 0.5052 - val_loss: 0.2565 - val_acc: 0.5094


In [34]:
modelSMALL.save_weights('model/modelSMALL.h5')

In [35]:
pred = modelSMALL.predict(x_test)
prediction = [float(x) for x in pred]

In [40]:
pred

array([[0.5931853],
       [0.5931853],
       [0.5931853],
       ...,
       [0.5931853],
       [0.5931853],
       [0.5931853]], dtype=float32)

In [36]:
def cut_half(x):
    if x < .5:
        return 0.
    else:
        return 1.

In [37]:
prediction = [cut_half(x) for x in prediction]

In [39]:
prediction[:1000]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [73]:
X = pd.DataFrame()
X["name"] = test_data
X["prediction"] = prediction
X.set_index("name")

ValueError: Length of values does not match length of index

In [37]:
df_submission

,prediction
name,
0.jpg,1
1.jpg,0
2.jpg,1
3.jpg,1
4.jpg,1
5.jpg,1
6.jpg,1
7.jpg,0
8.jpg,1


In [38]:
x_test.shape

(17056, 128, 128, 3)

In [ ]:
conv_size = 16
pool_size = (3,3)

model = Sequential()
model.add(Conv2D(conv_size, 
                 (3, 3), 
                 input_shape=input_shape, 
                 kernel_initializer=RN(0,1)))

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(conv_size, (3, 3), kernel_initializer=RN(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(conv_size, (3, 3),kernel_initializer=RN(1,1)))

model.add(Activation('sigmoid'))

model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])




In [41]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(XTRAIN)
epochs = 12
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(XTRAIN, train_labels, batch_size=32),
                    steps_per_epoch=len(XTRAIN) / 32, epochs=epochs, verbose=1)

# here's a more "manual" example



Epoch 1/12
251/250 [==============================] - 65s 260ms/step - loss: 0.2631 - acc: 0.5061
Epoch 2/12
251/250 [==============================] - 68s 273ms/step - loss: 0.2525 - acc: 0.4958
Epoch 3/12
251/250 [==============================] - 64s 255ms/step - loss: 0.2505 - acc: 0.4960
Epoch 4/12
251/250 [==============================] - 64s 256ms/step - loss: 0.2501 - acc: 0.5077
Epoch 5/12
251/250 [==============================] - 66s 261ms/step - loss: 0.2504 - acc: 0.4839
Epoch 6/12
251/250 [==============================] - 64s 253ms/step - loss: 0.2502 - acc: 0.5030
Epoch 7/12
159/250 [==================>...........] - ETA: 24s - loss: 0.2502 - acc: 0.5014

KeyboardInterrupt: 

In [ ]:
"""
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(XTRAIN, label, batch_size=32):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
"""



In [46]:
pred = modelSMALL.predict(x_test[:10])
prediction = [float(x) for x in pred]

In [48]:
prediction

[0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398,
 0.593185305595398]

In [45]:
x_test[:10]

array([[[[0.28627451, 0.29411765, 0.21176471],
         [0.32941176, 0.3372549 , 0.25490196],
         [0.38823529, 0.39607843, 0.31372549],
         ...,
         [0.16078431, 0.22352941, 0.18039216],
         [0.12941176, 0.19215686, 0.14901961],
         [0.10588235, 0.16862745, 0.1254902 ]],

        [[0.2627451 , 0.27058824, 0.18431373],
         [0.29803922, 0.30588235, 0.21960784],
         [0.34901961, 0.35686275, 0.2745098 ],
         ...,
         [0.16078431, 0.22352941, 0.18039216],
         [0.13333333, 0.19607843, 0.15294118],
         [0.10980392, 0.17254902, 0.12941176]],

        [[0.26666667, 0.2627451 , 0.18039216],
         [0.29019608, 0.28627451, 0.20392157],
         [0.32941176, 0.3254902 , 0.24313725],
         ...,
         [0.15686275, 0.21960784, 0.16862745],
         [0.13333333, 0.19607843, 0.14509804],
         [0.11372549, 0.17647059, 0.1254902 ]],

        ...,

        [[0.29411765, 0.26666667, 0.15294118],
         [0.29019608, 0.2627451 , 0.14901961]

In [1]:
rescale_factor = 255
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / rescale_factor,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / rescale_factor)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(size, size),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(size, size),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs)#,
    #validation_data=validation_generator,
    #validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

/Users/ppx/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 68218 images belonging to 2 classes.
Found 68218 images belonging to 2 classes.
Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 0.7073 - acc: 0.3750
Epoch 2/2
1/1 [==============================] - 0s 303ms/step - loss: 0.8141 - acc: 0.6250


In [2]:
import keras

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [27]:
data_train_path_target = "data_airbus_defi/train/"
data_test_path = "data_airbus_defi/test"
size = 128
batch_size = 16

In [28]:
train_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        data_train_path_target,
        target_size=(size, size),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        data_test_path,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')


Found 68218 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [120]:
modelSMALL = Sequential()
inputShape = (size, size, 3)
modelSMALL.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
modelSMALL.add(Activation("relu"))
modelSMALL.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
modelSMALL.add(Conv2D(50, (5, 5), padding="same"))
modelSMALL.add(Activation("relu"))
modelSMALL.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first (and only) set of FC => RELU layers
modelSMALL.add(Flatten())
modelSMALL.add(Dense(500))
modelSMALL.add(Activation("relu"))

# softmax classifier
modelSMALL.add(Dense(1))
modelSMALL.add(Activation("softmax"))


modelSMALL.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = modelSMALL.fit(
        XTRAIN, train_labels,
        steps_per_epoch=2000,
        epochs=50, verbose=1)

In [ ]:
history = modelSMALL.fit(
        XTRAIN, train_labels,
        steps_per_epoch=2000,
        epochs=50, verbose=1)